<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML321ENSkillsNetwork817-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Collaborative Filtering based Recommender System using K Nearest Neighbor**


Estimated time needed: **60** minutes


Collaborative filtering is probably the most commonly used recommendation algorithm, there are two main types of methods:
 - **User-based** collaborative filtering is based on the user similarity or neighborhood
 - **Item-based** collaborative filtering is based on similarity among items


They both work similarly, let's briefly explain how user-based collaborative filtering works.


User-based collaborative filtering looks for users who are similar. This is very similar to the user clustering method done previously; where we employed explicit user profiles to calculate user similarity. However, the user profiles may not be available, so how can we determine if two users are similar?


#### User-item interaction matrix


For most collaborative filtering-based recommender systems, the main dataset format is a 2-D matrix called the user-item interaction matrix. In the matrix,  its row is labeled as the user id/index and column labelled to be the item id/index, and the element `(i, j)` represents the rating of user `i` to item `j`.  

Below is a simple example of a user-item interaction matrix:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/module_4/images/user_item_matrix.png)


#### KNN-based collaborative filtering


As we can see from above, each row vector represents the rating history of a user and each column vector represents the users who rated the item. A user-item interaction matrix is usually very sparse as you can imagine one user very likely only interacts with a very small subset of items and one item is very likely to be interacted by a small subset of users.


Now to determine if two users are similar, we can simply calculate the similarities between their row vectors in the interaction matrix. Then based on the similarity measurements, we can find the `k` nearest neighbor as the similar users.


Item-based collaborative filtering works similarly, we just need to look at the user-item matrix vertically. Instead of finding similar users, we are trying to find similar items (courses). If two courses are enrolled by two groups of similar users, then we could consider the two items are similar and use the known ratings from the other users to predict the unknown ratings.


If we formulate the KNN based collaborative filtering,  the predicted rating of user $u$ to item $i$, $\hat{r}_{ui}$ is given by:


**User-based** collaborative filtering:


$$\hat{r}_{ui} = \frac{
\sum\limits_{v \in N^k_i(u)} \text{similarity}(u, v) \cdot r_{vi}}
{\sum\limits_{v \in N^k_i(u)} \text{similarity}(u, v)}$$


**Item-based** collaborative filtering:


$$\hat{r}_{ui} = \frac{
\sum\limits_{j \in N^k_u(i)} \text{similarity}(i, j) \cdot r_{uj}}
{\sum\limits_{j \in N^k_u(i)} \text{similarity}(i, j)}$$


Here $N^k_i(u)$ notates the nearest k neighbors of $u$.


Let's illustrate how the equation works using a simple example. From the above figure, suppose we want to predict the rating of `user6` to item `Machine Learning Capstone` course. After some similarity measurements, we found that k = 4 nearest neighbors: `user2, user3, user4, user5` with similarities in array ```knn_sims```:


In [1]:
import numpy as np
import math

In [2]:
# An example similarity array stores the similarity of user2, user3, user4, and user5 to user6
knn_sims = np.array([0.8, 0.92, 0.75, 0.83])

Also their rating on the `Machine Learning Capstone` course are:


In [3]:
# 2.0 means audit and 3.0 means complete the course
knn_ratings = np.array([3.0, 3.0, 2.0, 3.0])

So the predicted rating of `user6` to item `Machine Learning Capstone` course can be calculated as:


In [4]:
r_u6_ml =  np.dot(knn_sims, knn_ratings)/ sum(knn_sims)
r_u6_ml

np.float64(2.7727272727272725)

If we already know the true rating to be 3.0, then we get a prediction error RMSE (Rooted Mean Squared Error) as:


In [5]:
true_rating = 3.0
rmse = math.sqrt(true_rating - r_u6_ml) ** 2
rmse

0.22727272727272751

The predicted rating is around 2.7 (close to 3.0 with RMSE 0.22), which indicates that `user6` is also likely to complete the course `Machine Learning Capstone`. As such, we may recommend it to user6 with high confidence.


## Objectives


After completing this lab you will be able to:


* Perform KNN-based collaborative filtering on the user-item interaction matrix


----


### Load and exploring dataset


Let's first load our dataset, i.e., a user-item (learn-course) interaction matrix


In [6]:
import pandas as pd

In [7]:
rating_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-ML0321EN-Coursera/labs/v2/module_3/ratings.csv"
rating_df = pd.read_csv(rating_url)

In [8]:
rating_df.head()

,user,item,rating
0,1889878,CC0101EN,5
1,1342067,CL0101EN,3
2,1990814,ML0120ENv3,5
3,380098,BD0211EN,5
4,779563,DS0101EN,3


The dataset contains three columns, `user id` (learner), `item id`(course), and `rating`(enrollment mode).

Note that this matrix is presented as the dense or vertical form, and you may convert it to a sparse matrix using `pivot` :


In [9]:
rating_sparse_df = rating_df.pivot(index='user', columns='item', values='rating').fillna(0).reset_index().rename_axis(index=None, columns=None)
rating_sparse_df.head()

,user,AI0111EN,BC0101EN,BC0201EN,BC0202EN,BD0101EN,BD0111EN,BD0115EN,BD0121EN,BD0123EN,...,SW0201EN,TA0105,TA0105EN,TA0106EN,TMP0101EN,TMP0105EN,TMP0106,TMP107,WA0101EN,WA0103EN
0,2,0.0,4.0,0.0,0.0,5.0,4.0,0.0,5.0,3.0,...,0.0,5.0,0.0,4.0,0.0,3.0,3.0,0.0,5.0,0.0
1,4,0.0,0.0,0.0,0.0,5.0,3.0,4.0,5.0,3.0,...,0.0,4.0,0.0,0.0,0.0,3.0,3.0,0.0,3.0,3.0
2,5,3.0,5.0,5.0,0.0,4.0,0.0,0.0,0.0,3.0,...,0.0,0.0,4.0,4.0,4.0,4.0,4.0,5.0,0.0,3.0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Usually, the dense format is more preferred as it saves a lot of storage and memory space. While the benefit of the sparse matrix is it is in the nature matrix format and you could apply computations such as cosine similarity directly.


Next, you need to perform KNN-based collaborative filtering on the user-item interaction matrix.
You may choose one of the two following implementation options of KNN-based collaborative filtering.
- The first one is to use `scikit-surprise` which is a popular and easy-to-use Python recommendation system library.
- The second way is to implement it with standard `numpy`, `pandas`, and `sklearn`. You may need to write a lot of low-level implementation code along the way.


## Implementation Option 1: Use **Surprise** library (recommended)


*Surprise* is a Python sci-kit library for recommender systems. It is simple and comprehensive to build and test different recommendation algorithms.

First, let's install it:


Now we import required classes and methods


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Then, let's take a look at a code example how easily to perform KNN collaborative filtering on a sample movie review dataset, which contains about 100k movie ratings from users.


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

# 1. Load the movielens-100k dataset (u.data is tab-separated)
url = 'https://files.grouplens.org/datasets/movielens/ml-100k/u.data'
columns = ['user', 'item', 'rating', 'timestamp']
df = pd.read_csv(url, sep='\t', names=columns)

# 2. Split into Train and Test sets (25% test as per lab default)
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# 3. Create User-Item Matrix
# Rows are users, columns are movies
user_item_matrix = train_df.pivot(index='user', columns='item', values='rating').fillna(0)

# 4. Compute User Similarity Matrix (Cosine Similarity)
user_sim = cosine_similarity(user_item_matrix)
user_sim_df = pd.DataFrame(user_sim, index=user_item_matrix.index, columns=user_item_matrix.index)

# 5. Define KNN Prediction Logic
def predict_user_item(u_id, i_id, k=40):
    # Check if user or item exists in training set
    if i_id not in user_item_matrix.columns or u_id not in user_sim_df.index:
        return 3.5 # Global mean fallback

    # Get similarities and ratings for the target item
    sim_scores = user_sim_df[u_id]
    item_ratings = user_item_matrix[i_id]

    # Filter for users who actually rated this item
    idx = item_ratings[item_ratings > 0].index
    if len(idx) == 0:
        return 3.5

    relevant_sims = sim_scores[idx]
    relevant_ratings = item_ratings[idx]

    # Select Top-K neighbors
    top_k = relevant_sims.nlargest(k)
    top_k_ratings = relevant_ratings[top_k.index]

    if top_k.sum() == 0:
        return 3.5

    # Weighted average: sum(sim * rating) / sum(sim)
    return np.dot(top_k, top_k_ratings) / top_k.sum()

# 6. Calculate RMSE
print("Calculating predictions for MovieLens-100k...")
test_df = test_df.copy()
test_df['pred'] = test_df.apply(lambda x: predict_user_item(x['user'], x['item']), axis=1)
rmse = np.sqrt(mean_squared_error(test_df['rating'], test_df['pred']))

print(f"\nKNN Collaborative Filtering RMSE: {rmse:.4f}")

Calculating predictions for MovieLens-100k...

KNN Collaborative Filtering RMSE: 1.0111


As you can see, just a couple of lines and you can apply KNN collaborative filtering on the sample movie lens dataset. The main evaluation metric is `Root Mean Square Error (RMSE)` which is a very popular rating estimation error metric used in recommender systems as well as many regression model evaluations.


Now, let's load our own course rating dataset:


In [15]:
# Load the course rating dataset
rating_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-ML0321EN-Coursera/labs/v2/module_3/ratings.csv"
rating_df = pd.read_csv(rating_url)

# Display the first few rows to confirm the structure
rating_df.head()

,user,item,rating
0,1889878,CC0101EN,5
1,1342067,CL0101EN,3
2,1990814,ML0120ENv3,5
3,380098,BD0211EN,5
4,779563,DS0101EN,3


We split it into trainset and testset:


In [16]:
# Split the course rating dataset into 70% training and 30% testing
# This matches the common split used in the lab
train_df, test_df = train_test_split(rating_df, test_size=0.3, random_state=42)

then check how many users and items we can use to fit a KNN model:


In [17]:
print(f"Training rows: {len(train_df)}")
print(f"Testing rows: {len(test_df)}")

Training rows: 163314
Testing rows: 69992


### TASK: Perform KNN-based collaborative filtering on the user-item interaction matrix


_TODO: Fit the KNN-based collaborative filtering model using the trainset and evaluate the results using the testset:_


In [ ]:
## WRITE YOUR CODE HERE:


# - Define a KNNBasic() model
# Note there are some arguments such as:
# max_k and min_k, representing the max and min number of neighors for rating estimations
# sim_option, representing similarity measurement such as cosine and whether you want it to be user_based or items_based
# e.g., sim_option = {
#        'name': 'cosine', 'user_based': False,
#    }
#
# more KNN model hyperparamets can be found here:
# https://surprise.readthedocs.io/en/stable/knn_inspired.html
#
# You may try different hyperparamet combinations to see which one has the best performance


# - Train the KNNBasic model on the trainset, and predict ratings for the testset

# - Then compute RMSE


<details>
    <summary>Click here for Hints</summary>

* Create a model by calling `KNNBasic()` class.
* Fit it with `trainset` by using `model.fit(trainset)`.  
* Record predictions to the `testset`  by using `model.test(testset).
* Compute the accuracy by using `accuracy.rmse(predictions)`


To learn more detailed usages about _Surprise_ library, visit its website from [here](https://surprise.readthedocs.io/en/stable/getting_started.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML321ENSkillsNetwork817-2022-01-01)


## Implementation Option 2: Use `numpy`, `pandas`, and `sklearn`


If you do not prefer the one-stop Suprise solution and want more hardcore coding practices, you may implement the KNN model using `numpy`, `pandas`, and possibly `sklearn`:


In [18]:
## WRITE YOUR CODE HERE:

## One solution could be:
## - Calculate the similarity between two users using their rating history (the row vectors of interaction matrix)

## - Build a similarity matrix for each pair of users with the training dataset

## - For each user, find its k nearest neighbors in the sim matrix

## - For each rating in the test dataset, estimate its rating using the KNN collaborative filtering equations shown before

## - Calculate RMSE for the entire test dataset

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

# 1. Build the interaction matrix (User-Item Matrix)
# Rows are users, columns are items, values are ratings
user_item_matrix = train_df.pivot(index='user', columns='item', values='rating').fillna(0)

# 2. Calculate the similarity matrix for each pair of users
# We use cosine_similarity on the row vectors
user_sim_matrix = cosine_similarity(user_item_matrix)
user_sim_df = pd.DataFrame(user_sim_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

# 3. Define the prediction logic using KNN equations
def predict_rating_hardcore(user_id, item_id, k=40):
    # If the user or item was not seen in training, we use the training mean as a baseline
    if user_id not in user_sim_df.index or item_id not in user_item_matrix.columns:
        return train_df['rating'].mean()

    # Get all users who have rated this specific item
    users_who_rated_item = user_item_matrix[user_item_matrix[item_id] > 0].index

    # Get similarity scores between our target user and those users
    sim_scores = user_sim_df.loc[user_id, users_who_rated_item]

    # If no one in the training set rated this item, return baseline
    if sim_scores.empty:
        return train_df['rating'].mean()

    # 4. Find the k nearest neighbors for this specific item
    # We take the top k highest similarity scores
    top_k_neighbors = sim_scores.nlargest(k)

    # Get the ratings from these top k neighbors for the item
    neighbor_ratings = user_item_matrix.loc[top_k_neighbors.index, item_id]

    # Calculate the weighted average: sum(similarity * rating) / sum(similarities)
    sum_sims = top_k_neighbors.sum()
    if sum_sims == 0:
        return train_df['rating'].mean()

    prediction = np.dot(top_k_neighbors, neighbor_ratings) / sum_sims
    return prediction

# 5. For each rating in the test dataset, estimate its rating
print("Estimating ratings for the test dataset...")
test_df_results = test_df.copy()
test_df_results['predicted_rating'] = test_df_results.apply(
    lambda row: predict_rating_hardcore(row['user'], row['item'], k=40), axis=1
)

# 6. Calculate RMSE for the entire test dataset
actual_ratings = test_df_results['rating']
predicted_ratings = test_df_results['predicted_rating']

rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
print(f"\nManual KNN Implementation RMSE: {rmse:.4f}")

# Display sample results
test_df_results[['user', 'item', 'rating', 'predicted_rating']].head(10)

Estimating ratings for the test dataset...

Manual KNN Implementation RMSE: 0.8584


,user,item,rating,predicted_rating
173735,1232762,DS0103EN,3,3.721748
20280,1340444,CO0101EN,3,3.870420
201508,1310442,BD0211EN,3,3.747957
170108,1141000,BD0212EN,5,3.715188
187957,1802923,DS0101EN,4,3.836123
24235,435270,RP0151EN,5,3.999620
97215,1592207,SC0103EN,4,4.001239
215994,1452915,CP0101EN,4,3.619062
220019,1827156,DS0101EN,4,3.441874
111534,1113405,DS0301EN,3,3.695530


## Summary



In this lab, you have learned and implemented KNN-based collaborative filtering. It is probably the simplest but very effective and intuitive collaborative filtering algorithm. Since it is based on KNN, it inherits the main characteristics of KNN such as memory-intensive because you need to maintain a huge similarity matrix among users or items. In the future labs, we will learn other types of collaborative filtering which do not rely on such a huge similarity matrix to make rating predictions.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


```toggle## Change Log
```


```toggle|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
```
```toggle|-|-|-|-|
```
```toggle|2021-10-25|1.0|Yan|Created the initial version|
```
